Installation of necessary packages (and creation of a conda env):
```
./install_main.sh 
```

Process results obtained via running command:
```
bash -i run_mnli_main.sh
```

In [1]:
SEED_QTY=3
OUTPUT_PREF='results_mnli/output_res_'

In [2]:
#sync_step_arr = [1, 2, 4, 8, 16]
sync_step_arr = [1, 2, 4, 8, 16, 32, 64, 128, 256]

res_keys = ['1gpu'] + \
        [f'accum_steps_{k}' for k in sync_step_arr] + \
        [f'nosync_steps_{k}' for k in sync_step_arr]

In [3]:
import json

res_dict = {k : [] for k in res_keys}

for train_qty in [4000, 40000]:
    for rk in res_keys:
        one_res = {}
        
        for seed in range(SEED_QTY):
            tmp = json.load(open(f'{OUTPUT_PREF}{train_qty}_{rk}/{seed}/all_results.json'))
            if seed == 0:
                for mk in tmp.keys():
                    one_res[mk] = []

            for mk in tmp.keys():
                one_res[mk].append(tmp[mk])
                 
        res_dict[rk].append(one_res)
    

In [4]:
res_dict['1gpu']

[{'eval_accuracy': [0.5706875508543532,
   0.7505085435313262,
   0.3295362082994304],
  'train_time': [74.01993560791016, 73.50326371192932, 74.1038613319397]},
 {'eval_accuracy': [0.8176362896663955, 0.817432872253865, 0.8233319772172498],
  'train_time': [732.0480329990387, 726.9911198616028, 727.3452048301697]}]

In [5]:
tmp

{'eval_accuracy': 0.7955560064935064, 'train_time': 189.10353302955627}

In [12]:
import numpy as np

res_dict_str = {k : [] for k in res_keys}

for cidx, train_qty in enumerate([4000, 40000]):
    for rk in res_keys:
        speed_up = round(np.mean(res_dict['1gpu'][cidx]['train_time']) / np.mean(res_dict[rk][cidx]['train_time']), 1)
        acc = str(round(np.mean(res_dict[rk][cidx]['eval_accuracy']), 1)) +'/' +str(round(np.max(res_dict[rk][cidx]['eval_accuracy']), 1))
        #print(rk, speed_up, f1)
        if rk != '1gpu':
            res_dict_str[rk].append(f'{acc} {speed_up}x')
        else:
            res_dict_str[rk].append(f'{acc}')  
            
            
res_dict_str

{'1gpu': ['0.6/0.8', '0.8/0.8'],
 'accum_steps_1': ['0.6/0.7 1.3x', '0.8/0.8 1.5x'],
 'accum_steps_2': ['0.5/0.6 2.8x', '0.8/0.8 2.9x'],
 'accum_steps_4': ['0.4/0.4 5.1x', '0.8/0.8 5.8x'],
 'accum_steps_8': ['0.3/0.4 9.0x', '0.7/0.7 11.2x'],
 'accum_steps_16': ['0.3/0.4 14.4x', '0.5/0.6 21.7x'],
 'accum_steps_32': ['0.3/0.4 20.8x', '0.4/0.5 41.2x'],
 'accum_steps_64': ['0.3/0.4 21.3x', '0.4/0.4 73.6x'],
 'accum_steps_128': ['0.3/0.4 21.5x', '0.3/0.4 108.7x'],
 'accum_steps_256': ['0.3/0.4 20.8x', '0.3/0.4 137.1x'],
 'nosync_steps_1': ['0.6/0.6 1.2x', '0.8/0.8 1.2x'],
 'nosync_steps_2': ['0.6/0.6 1.8x', '0.8/0.8 1.8x'],
 'nosync_steps_4': ['0.5/0.6 2.5x', '0.8/0.8 2.4x'],
 'nosync_steps_8': ['0.5/0.6 3.0x', '0.8/0.8 3.0x'],
 'nosync_steps_16': ['0.5/0.6 3.4x', '0.8/0.8 3.4x'],
 'nosync_steps_32': ['0.5/0.5 3.7x', '0.8/0.8 3.6x'],
 'nosync_steps_64': ['0.5/0.5 3.8x', '0.8/0.8 3.8x'],
 'nosync_steps_128': ['0.5/0.5 3.7x', '0.8/0.8 3.8x'],
 'nosync_steps_256': ['0.5/0.5 3.8x', '0.8/0.8 3.9

In [13]:
import pandas as pd
df=pd.DataFrame.from_dict(res_dict_str)

In [14]:
df

,1gpu,accum_steps_1,accum_steps_2,accum_steps_4,accum_steps_8,accum_steps_16,accum_steps_32,accum_steps_64,accum_steps_128,accum_steps_256,nosync_steps_1,nosync_steps_2,nosync_steps_4,nosync_steps_8,nosync_steps_16,nosync_steps_32,nosync_steps_64,nosync_steps_128,nosync_steps_256
0,0.6/0.8,0.6/0.7 1.3x,0.5/0.6 2.8x,0.4/0.4 5.1x,0.3/0.4 9.0x,0.3/0.4 14.4x,0.3/0.4 20.8x,0.3/0.4 21.3x,0.3/0.4 21.5x,0.3/0.4 20.8x,0.6/0.6 1.2x,0.6/0.6 1.8x,0.5/0.6 2.5x,0.5/0.6 3.0x,0.5/0.6 3.4x,0.5/0.5 3.7x,0.5/0.5 3.8x,0.5/0.5 3.7x,0.5/0.5 3.8x
1,0.8/0.8,0.8/0.8 1.5x,0.8/0.8 2.9x,0.8/0.8 5.8x,0.7/0.7 11.2x,0.5/0.6 21.7x,0.4/0.5 41.2x,0.4/0.4 73.6x,0.3/0.4 108.7x,0.3/0.4 137.1x,0.8/0.8 1.2x,0.8/0.8 1.8x,0.8/0.8 2.4x,0.8/0.8 3.0x,0.8/0.8 3.4x,0.8/0.8 3.6x,0.8/0.8 3.8x,0.8/0.8 3.8x,0.8/0.8 3.9x


In [ ]:
#df.to_csv('results.tsv')

In [ ]:
#!cat results.tsv